In [37]:
import numpy as np
from scipy.sparse import coo_matrix, vstack, linalg
from scipy.sparse.linalg import cg
import mcubes
import open3d as o3d
import trimesh

In [38]:
def generate_point_cloud_with_normals(mesh, num_points=1000):
    # Randomly sample points from the mesh
    random_indices = np.random.choice(mesh.vertices.shape[0], num_points, replace=False)
    points = mesh.vertices[random_indices]
    normals = mesh.vertex_normals[random_indices]
    pc_mesh = trimesh.Trimesh(vertices=points, vertex_normals=normals)
    return pc_mesh

def save_point_cloud(mesh, mesh_file_name):
    # Save the point cloud as numpy array file, because Trimesh cannot preserve normals when saving point cloud 
    point_cloud_filename = "PointCloud_" + mesh_file_name.split('.')[0]
    np.savez(point_cloud_filename, vertices=mesh.vertices, vertex_normals=mesh.vertex_normals)

def load_point_cloud(mesh_file_name):
    # Load the point cloud from numpy array file
    input_point_cloud_filename = "PointCloud_" + mesh_file_name.split('.')[0] + ".npz"
    data = np.load(input_point_cloud_filename)
    return trimesh.Trimesh(vertices=data['vertices'], vertex_normals=data['vertex_normals'])

In [39]:
mesh_file_name = "bun_zipper_res3.ply"
sample_num_points = 1500

if True:
    # Generate point cloud from mesh
    original_mesh = trimesh.load_mesh(mesh_file_name)
    point_cloud = generate_point_cloud_with_normals(original_mesh, num_points=sample_num_points)
    save_point_cloud(point_cloud, mesh_file_name)

In [40]:
input_point_cloud = load_point_cloud(mesh_file_name)
num_cubes = np.array([64, 64, 64])
padding = 4

input_points = input_point_cloud.vertices
input_normals = -input_point_cloud.vertex_normals # Reverse the normals because they are pointing inwards

def get_voxel_grid(vertices, num_cubes, padding):
    bounding_box = np.max(vertices, axis=0) - np.min(vertices, axis=0)
    print(bounding_box) # Bounding box means the maximum coordinates of the bounding box
    voxel_size = np.max(bounding_box) / num_cubes
    origin = np.min(vertices, axis=0) - padding * voxel_size
    voxel_num = num_cubes + 2 * padding
    return origin, voxel_size, voxel_num

origin, voxel_size, voxel_num = get_voxel_grid(input_points, num_cubes, padding)


print(origin) # Origin means the minimum coordinates of the bounding box
print(voxel_size) # Voxel size means the size of each voxel after dividing the bounding box into num_cubes
print(voxel_num) # Voxel resolution means the number of voxels in each dimension

[0.1552989  0.15133359 0.1201372 ]
[-0.10407048  0.02377322 -0.07137828]
[0.00242655 0.00242655 0.00242655]
[72 72 72]


In [41]:
def fd_grad(num_voxel, size_voxel):
    # This is used to calculate the gradient of any scalar field defined on the grid
    nx, ny, nz = num_voxel
    hx, hy, hz = size_voxel

    idx_primary_grid = np.arange(nx * ny * nz).reshape((nx, ny, nz))

    num_staggered_grid_x = (nx - 1) * ny * nz
    idx_col_x = np.concatenate((idx_primary_grid[1:, ...].flatten(), idx_primary_grid[:-1, :, :].flatten()))
    row_idx_x = np.arange(num_staggered_grid_x)
    row_idx_x = np.tile(row_idx_x, 2)
    data_term_x = [1 / hx] * num_staggered_grid_x + [-1 / hx] * num_staggered_grid_x
    Dx = coo_matrix((data_term_x, (row_idx_x, idx_col_x)), shape=(num_staggered_grid_x, nx * ny * nz))

    num_staggered_grid_y = nx * (ny - 1) * nz
    idx_col_y = np.concatenate((idx_primary_grid[:, 1:, :].flatten(), idx_primary_grid[:, :-1, :].flatten()))
    row_idx_y = np.arange(num_staggered_grid_y)
    row_idx_y = np.tile(row_idx_y, 2)
    data_term_y = [1 / hy] * num_staggered_grid_y + [-1 / hy] * num_staggered_grid_y
    Dy = coo_matrix((data_term_y, (row_idx_y, idx_col_y)), shape=(num_staggered_grid_y, nx * ny * nz))

    num_staggered_grid_z = nx * ny * (nz - 1)
    idx_col_z = np.concatenate((idx_primary_grid[:, :, 1:].flatten(), idx_primary_grid[:, :, :-1].flatten()))
    row_idx_z = np.arange(num_staggered_grid_z)
    row_idx_z = np.tile(row_idx_z, 2)
    data_term_z = [1 / hz] * num_staggered_grid_z + [-1 / hz] * num_staggered_grid_z
    Dz = coo_matrix((data_term_z, (row_idx_z, idx_col_z)), shape=(num_staggered_grid_z, nx * ny * nz))


    G = vstack((Dx, Dy, Dz))
    return G

G = fd_grad(voxel_num, voxel_size)

In [42]:
def trilinear_interpolation_weights(n_cube, corner, points, s_cube, direction=None):
    nx, ny, nz = n_cube
    hx, hy, hz = s_cube
    
    # grid coordinates / indices
    # minusing corner's coordinates makes the samples started from origin and the floor of them are the cubes' index
    # they are in. the auther call it as "grid coordinates/indices"

    x0 = np.floor((points[:, 0] - corner[0]) / hx).astype(int)  # (N, )
    y0 = np.floor((points[:, 1] - corner[1]) / hy).astype(int)  # (N, )
    z0 = np.floor((points[:, 2] - corner[2]) / hz).astype(int)  # (N, )
    x1 = x0 + 1  # (N, )
    y1 = y0 + 1  # (N, )
    z1 = z0 + 1  # (N, )

    # the coordinates of samples in their local cubes, the numeral values of them are the percentages along each axis.
    xd = (points[:, 0] - corner[0]) / hx - x0  # (N, )
    yd = (points[:, 1] - corner[1]) / hy - y0  # (N, )
    zd = (points[:, 2] - corner[2]) / hz - z0  # (N, )

    # data terms for the trilinear interpolation weight matrix
    weight_000 = (1 - xd) * (1 - yd) * (1 - zd)
    weight_100 = xd * (1 - yd) * (1 - zd)
    weight_010 = (1 - xd) * yd * (1 - zd)
    weight_110 = xd * yd * (1 - zd)
    weight_001 = (1 - xd) * (1 - yd) * zd
    weight_101 = xd * (1 - yd) * zd
    weight_011 = (1 - xd) * yd * zd
    weight_111 = xd * yd * zd
    data_term = np.concatenate((weight_000,
                                weight_100,
                                weight_010,
                                weight_110,
                                weight_001,
                                weight_101,
                                weight_011,
                                weight_111))

    row_idx = np.arange(points.shape[0])
    row_idx = np.tile(row_idx, 8)

    if direction == "x":
        num_grids = (nx - 1) * ny * nz
        staggered_grid_idx = np.arange((nx - 1) * ny * nz).reshape((nx - 1, ny, nz))
    elif direction == "y":
        num_grids = nx * (ny - 1) * nz
        staggered_grid_idx = np.arange(nx * (ny - 1) * nz).reshape((nx, ny - 1, nz))
    elif direction == "z":
        num_grids = nx * ny * (nz - 1)
        staggered_grid_idx = np.arange(nx * ny * (nz - 1)).reshape((nx, ny, nz - 1))
    else:
        num_grids = nx * ny * nz
        staggered_grid_idx = np.arange(nx * ny * nz).reshape((nx, ny, nz))

    col_idx_000 = staggered_grid_idx[x0, y0, z0]
    col_idx_100 = staggered_grid_idx[x1, y0, z0]
    col_idx_010 = staggered_grid_idx[x0, y1, z0]
    col_idx_110 = staggered_grid_idx[x1, y1, z0]
    col_idx_001 = staggered_grid_idx[x0, y0, z1]
    col_idx_101 = staggered_grid_idx[x1, y0, z1]
    col_idx_011 = staggered_grid_idx[x0, y1, z1]
    col_idx_111 = staggered_grid_idx[x1, y1, z1]
    col_idx = np.concatenate((col_idx_000,
                                col_idx_100,
                                col_idx_010,
                                col_idx_110,
                                col_idx_001,
                                col_idx_101,
                                col_idx_011,
                                col_idx_111))

    W = coo_matrix((data_term, (row_idx, col_idx)), shape=(points.shape[0], num_grids))
    return W

print(voxel_num, origin, input_points, voxel_size)
Wx = trilinear_interpolation_weights(voxel_num, origin, input_points, voxel_size, direction="x")
Wy = trilinear_interpolation_weights(voxel_num, origin, input_points, voxel_size, direction="y")
Wz = trilinear_interpolation_weights(voxel_num, origin, input_points, voxel_size, direction="z")
W = trilinear_interpolation_weights(voxel_num, origin, input_points, voxel_size)

[72 72 72] [-0.10407048  0.02377322 -0.07137828] [[-0.0205725   0.123523    0.0265579 ]
 [ 0.0602205   0.0643718   0.00864139]
 [-0.0599262   0.0622966  -0.00429285]
 ...
 [-0.0523435   0.0697862  -0.0145984 ]
 [-0.0192899   0.0641483   0.0503928 ]
 [ 0.0257452   0.0459137   0.0381185 ]] [0.00242655 0.00242655 0.00242655]


In [43]:
V = np.concatenate(
    [
        Wx.T @ input_normals[:, 0],
        Wy.T @ input_normals[:, 1],
        Wz.T @ input_normals[:, 2],
    ]
)

In [44]:
g, _ = cg(G.T @ G, G.T @ V, maxiter=2000, tol=1e-5)
g -= np.mean(W @ g)
g_field = g.reshape(voxel_num)
vertices, triangles = mcubes.marching_cubes(g_field, 0)

In [45]:
vertices[:, 0] = vertices[:, 0] * voxel_size[0] + origin[0]
vertices[:, 1] = vertices[:, 1] * voxel_size[1] + origin[1]
vertices[:, 2] = vertices[:, 2] * voxel_size[2] + origin[2]

output_mesh = trimesh.Trimesh(vertices=vertices, faces=triangles)
output_mesh.export("output_mesh.ply")

b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 13866\nproperty float x\nproperty float y\nproperty float z\nelement face 27728\nproperty list uchar int vertex_indices\nend_header\nD\x9e\xbd\xbd&\x80\xf2=3\x04^<\xddI\xbc\xbdz\xf0\xee=3\x04^<\xddI\xbc\xbd&\x80\xf2=2\xa9@<\xa9\xb4\xbd\xbd&\x80\xf2=\xef\xe2\x82<\xddI\xbc\xbd\x05z\xee=\xef\xe2\x82<\x8f\x7f\xbe\xbd&\x80\xf2=\xc5\xc3\x96<\xddI\xbc\xbd\x02\xf9\xee=\xc5\xc3\x96<\xf4\x97\xbe\xbd&\x80\xf2=\x9a\xa4\xaa<\xddI\xbc\xbd\xe8\xaa\xef=\x9a\xa4\xaa<\xddI\xbc\xbd&\x80\xf2=\xec\x95\xb5<Q\x0f\xbd\xbd[x\xf7=\x88B6<\xddI\xbc\xbd$\x89\xf5=\x88B6<\xddI\xbc\xbd[x\xf7=\xf6\x0e,<\xe3u\xbe\xbd[x\xf7=3\x04^<\\\x0b\xbf\xbd[x\xf7=\xef\xe2\x82<\xf6\xf6\xbe\xbd[x\xf7=\xc5\xc3\x96<\x10\xbd\xbe\xbd[x\xf7=\x9a\xa4\xaa<u\xf6\xbd\xbd[x\xf7=p\x85\xbe<\xddI\xbc\xbdM/\xf5=p\x85\xbe<\x9c\x82\xbe\xbd[x\xf7=Ef\xd2<\xddI\xbc\xbd\x03o\xf6=Ef\xd2<\x18\xc6\xbd\xbd[x\xf7=\x1bG\xe6<\xddI\xbc\xbd\x84\xcc\xf6=\x1bG\xe6<\xd

In [46]:
def build_open3d_octree(points, depth):
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    
    octree = o3d.geometry.Octree(max_depth=depth)
    octree.convert_from_point_cloud(point_cloud, size_expand=0.01)
    return octree


depth = 8  # Depth of the octree
points = input_points
normals = input_normals

octree = build_open3d_octree(points, depth)
octree.traverse(lambda node, _node_info: print(node))

OctreeInternalPointNode with 8 non-empty child nodes and 1500 points
OctreeInternalPointNode with 4 non-empty child nodes and 191 points
OctreeInternalPointNode with 2 non-empty child nodes and 19 points
OctreeInternalPointNode with 3 non-empty child nodes and 9 points
OctreeInternalPointNode with 1 non-empty child nodes and 2 points
OctreeInternalPointNode with 2 non-empty child nodes and 2 points
OctreeInternalPointNode with 1 non-empty child nodes and 1 points
OctreeInternalPointNode with 1 non-empty child nodes and 1 points
OctreePointColorLeafNode with color [0, 0, 0] containing 1 points.
OctreeInternalPointNode with 1 non-empty child nodes and 1 points
OctreeInternalPointNode with 1 non-empty child nodes and 1 points
OctreePointColorLeafNode with color [0, 0, 0] containing 1 points.
OctreeInternalPointNode with 5 non-empty child nodes and 5 points
OctreeInternalPointNode with 1 non-empty child nodes and 1 points
OctreeInternalPointNode with 1 non-empty child nodes and 1 points
Oc

In [47]:
def use_octree_for_reconstruction(octree, points, normals):
    pass